# New Section

In [3]:
!pip install datasets

In [4]:
from datasets import load_dataset
ds = load_dataset("knkarthick/dialogsum")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [6]:
ds['train'][1]['dialogue']
# First dilogue from the train part of the dataset

"#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little."

In [7]:
ds['train'][1]['summary']
# First summary from the train part of the dataset

'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.'

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("cammy/distilbart-cnn-12-6-finetuned-weaksup-1000")
model = AutoModelForSeq2SeqLM.from_pretrained("cammy/distilbart-cnn-12-6-finetuned-weaksup-1000")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [9]:
##Tokenization [Strings to Vector to Preprocess]

def preprocess_function(batch):
  source = batch['dialogue']
  target = batch['summary']
  source_ids = tokenizer(source, truncation=True, max_length = 128,padding = 'max_length')
  target_ids = tokenizer(target, truncation=True, max_length = 128,padding = 'max_length')

  labels = target_ids['input_ids']
  labels = [[(label if label != tokenizer.pad_token_type_id else -100)
              for label in labels_example] for labels_example in labels]#removing teh padded zeroes or token with -100 so that it can identify which labels to ignore

  return {
      "input_ids" : source_ids['input_ids'],   #inputs indices from the source_ids #giving the vectorized form of the tokens

      "attention_mask" : source_ids['attention_mask'],
      "labels" : labels
  }

In [10]:
## Mapping this function to our dataframe
ds_source = ds.map(preprocess_function, batched = True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [11]:
##Training Arguments

from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir = "/content",
    per_device_train_batch_size = 8,
    num_train_epochs = 2,
    remove_unused_columns = True
)

In [12]:
##Creating teh trainer  objects using Trainer Model

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = ds_source['train'],
    eval_dataset = ds_source['test']
)


In [13]:
##Training the model
trainer.train()
#5e5a0bcc46819b6d7bf14e5253c55b35d33b7d17

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: singhanushka0214 (singhanushka0214-kiit-deemed-to-be-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
500,0.679900
1000,0.472900
1500,0.458000
2000,0.359800
2500,0.341500
3000,0.335900


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3116, training_loss=0.43822360497843166, metrics={'train_runtime': 2669.1319, 'train_samples_per_second': 9.336, 'train_steps_per_second': 1.167, 'total_flos': 4821746146344960.0, 'train_loss': 0.43822360497843166, 'epoch': 2.0})

In [14]:
##Evaluating the results
eval_results = trainer.evaluate()
eval_results

{'eval_loss': 0.3962234556674957,
 'eval_runtime': 38.9542,
 'eval_samples_per_second': 38.507,
 'eval_steps_per_second': 4.826,
 'epoch': 2.0}

In [15]:
##Saving the model
model.save_pretrained('/content/model_dir')
tokenizer.save_pretrained('content/model_dir')

('content/model_dir/tokenizer_config.json',
 'content/model_dir/special_tokens_map.json',
 'content/model_dir/vocab.json',
 'content/model_dir/merges.txt',
 'content/model_dir/added_tokens.json',
 'content/model_dir/tokenizer.json')

In [16]:
#Checking with the unseen data with saved model & tokenizer instead of HuggingFace
# tokenizer = AutoTokenizer.from_pretrained('/content/model_dir')
# model = AutoModelForSeq2SeqLM.from_pretrained('/content/model_dir')

In [17]:
# Function to input the new unseen data
def summarize(blog_post):
#tokenizing the input blog post
  inputs = tokenizer(blog_post, max_length = 1024, truncation = True, return_tensors = 'pt')
  # Move the input tensor to the same device as the model
  inputs['input_ids'] = inputs['input_ids'].to(model.device)
##Generate the summary
  summary_ids = model.generate(inputs['input_ids'], max_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

##decode the summary as it will be created in the forms of tokens
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens = True)
  return summary

In [18]:
##inputing the blog post
blog_post = """ As Yogi Berra famously said, it’s tough to make predictions, especially about the future. But had the baseball legend spent any time observing the UN climate negotiations, he could have safely predicted that climate finance will prove to be a key sticking point at COP29 in Baku at the end of this year.

‘Who will pay and how much?’ are perennial questions at the climate talks, but this year, the discussions about climate finance will be especially prominent. At COP29, Parties to the Paris Agreement must negotiate a new climate finance goal, to replace the existing commitment from 2009 for developed countries to provide US$100 billion climate finance annually from 2020 to 2025 - a commitment that only in 2022 was starting to be fulfilled, according to a recent OECD report.

It is vital that the forthcoming Bonn Climate Change Conference sends the right political signals, and lays the procedural and technical groundwork for an ambitious climate finance deal in Baku.

A pressing need

With global warming already destabilising the climate and devastating people’s lives and livelihoods, the need for finance to reduce greenhouse gas emissions and to adapt to a warming world has never been more pressing.

The sums involved are large. The Paris Agreement’s Global Stocktake process estimates that US$5.8-5.9 trillion is required to implement Nationally Determined Contributions (NDCs) in developing countries up to 2030. They will require US$215-387 billion annually over this period for adaptation. Investments of US$1.5 trillion in renewable energy are required worldwide every year up until 2030, according to IRENA.

But these sums are also affordable and beneficial for developed countries. They should be seen in the context of ongoing investments in energy and other infrastructure: around US$2.3 trillion was invested in energy infrastructure in 2023, of which US$1.74 trillion was in clean energy. These investments will generate strong returns for their investors and reduce the costs for energy consumers.

And, crucially, they should also be seen in the context of the alternative. The latest research estimates that the world economy is already set to face a 19% income reduction within the next 26 years based on the levels of warming we have already locked in. The more we delay and the more the planet heats, the greater the economic costs will be.

Laying the foundations for a new finance goal

While financial resources are beginning to flow, they are not flowing fast enough, and certainly not flowing to those developing countries where need is greatest and access to finance is most challenging.

The UN climate framework provides mechanisms that can enable those flows of climate finance. Back in 2015, parties at the climate talks agreed to establish a “new collective quantified goal” (NCQG) for climate finance. They agreed that the NCQG would be set prior to 2025.

The  ultimate size of the NCQG will be a product of the negotiations, but Parties have agreed it must be a significant increase from the floor of US$100 billion annually. For WWF, it must be needs-based and sufficiently ambitious to meet the scale of the challenge we face, and immediately accessible to help countries that are already facing the chaos of a destabilised climate system.

While developed countries are expected to provide financial and technical support, developing countries also have a role to play. Parties are due to submit revised NDCs in 2025, presenting how they plan to reduce emissions and adapt to climate change. Developing countries have the opportunity to use their NDCs to set out how international climate finance can support them and increase their ambition. To do this, they need to know the finance will be forthcoming."""

summary = summarize(blog_post)
summary

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1427: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (40). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


'The need for finance to reduce greenhouse gas emissions and to adapt to a warming world has never been more pressing. The UN climate framework provides mechanisms that can enable those flows of climate finance.'

In [28]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ff1e97ccbed661184c5608f803cfc5d1d2d2e6a450978bed959e3c6b45b4f0e9
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [29]:
from rouge_score import rouge_scorer #importing rouge scorer
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [36]:
# ##Evaluating the model
from rouge_score import rouge_scorer, scoring

def evaluate_model(validation_data):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    aggregator = scoring.BootstrapAggregator()  # To aggregate scores
    predictions = []
    references = []

    for data in validation_data:
        text = data['dialogue']
        reference_summary = data['summary']

        # Generate summary
        generated_summary = summarize(text)

        # Compute ROUGE scores
        scores = scorer.score(reference_summary, generated_summary)
        aggregator.add_scores(scores)

    # Get aggregated ROUGE scores
    result = aggregator.aggregate()
    return result

In [37]:
# ##Evaluate the model on the validation dataset
validation_scores = evaluate_model(ds['validation'])

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1427: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (40). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.380862240220653, recall=0.5359173684140287, fmeasure=0.42750381030342344), mid=Score(precision=0.3912760916269588, recall=0.5516378387076546, fmeasure=0.43786720772057286), high=Score(precision=0.40238129724280425, recall=0.5689768773499272, fmeasure=0.447279476635671)), 'rouge2': AggregateScore(low=Score(precision=0.1732875444395108, recall=0.25831624779908313, fmeasure=0.19793779508636947), mid=Score(precision=0.18397918062765994, recall=0.27485188614795286, fmeasure=0.2099405294928648), high=Score(precision=0.1946055280585611, recall=0.2920192828879535, fmeasure=0.22166020407530662)), 'rougeL': AggregateScore(low=Score(precision=0.30432016111383, recall=0.43379445707131775, fmeasure=0.34441123119053435), mid=Score(precision=0.3142210237212749, recall=0.4493910380403825, fmeasure=0.35361589373014524), high=Score(precision=0.32486456148060205, recall=0.4653710753695263, fmeasure=0.3637845527012519))}


In [38]:
validation_scores

{'rouge1': AggregateScore(low=Score(precision=0.380862240220653, recall=0.5359173684140287, fmeasure=0.42750381030342344), mid=Score(precision=0.3912760916269588, recall=0.5516378387076546, fmeasure=0.43786720772057286), high=Score(precision=0.40238129724280425, recall=0.5689768773499272, fmeasure=0.447279476635671)),
 'rouge2': AggregateScore(low=Score(precision=0.1732875444395108, recall=0.25831624779908313, fmeasure=0.19793779508636947), mid=Score(precision=0.18397918062765994, recall=0.27485188614795286, fmeasure=0.2099405294928648), high=Score(precision=0.1946055280585611, recall=0.2920192828879535, fmeasure=0.22166020407530662)),
 'rougeL': AggregateScore(low=Score(precision=0.30432016111383, recall=0.43379445707131775, fmeasure=0.34441123119053435), mid=Score(precision=0.3142210237212749, recall=0.4493910380403825, fmeasure=0.35361589373014524), high=Score(precision=0.32486456148060205, recall=0.4653710753695263, fmeasure=0.3637845527012519))}

In [39]:
!pip install nltk bert-score datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 924.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [46]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from bert_score import score
from datasets import load_dataset

# Download NLTK tokenizer data
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [44]:
##Using BLEU Metrics

def compute_bleu(reference, prediction):
    reference_tokens = [nltk.word_tokenize(reference.lower())]
    prediction_tokens = nltk.word_tokenize(prediction.lower())

    # Apply smoothing to avoid zero scores for short texts
    smoothie = SmoothingFunction().method1
    return sentence_bleu(reference_tokens, prediction_tokens, smoothing_function=smoothie)

def evaluate_bleu(dataset_split):
    bleu_scores = []
    for data in ds[dataset_split]:
        reference_summary = data["summary"]
        generated_summary = summarize(data["dialogue"])  # Assuming you have a summarization function

        bleu_scores.append(compute_bleu(reference_summary, generated_summary))

    return sum(bleu_scores) / len(bleu_scores)

# Compute BLEU for validation set
bleu_score_value = evaluate_bleu("validation")
print(f"BLEU Score (Validation): {bleu_score_value:.4f}")

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1427: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (40). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


BLEU Score (Validation): 0.1727


In [47]:
##METEOR SCORE EVALUATION
def compute_meteor(reference, prediction):
    reference_tokens = [nltk.word_tokenize(reference.lower())]
    prediction_tokens = nltk.word_tokenize(prediction.lower())
    return meteor_score(reference_tokens, prediction_tokens)

def evaluate_meteor(dataset_split):
    meteor_scores = []
    for data in ds[dataset_split]:
        reference_summary = data["summary"]
        generated_summary = summarize(data["dialogue"])

        meteor_scores.append(compute_meteor(reference_summary, generated_summary))

    return sum(meteor_scores) / len(meteor_scores)

# Compute METEOR for validation set
meteor_score_value = evaluate_meteor("validation")
print(f"METEOR Score (Validation): {meteor_score_value:.4f}")

METEOR Score (Validation): 0.4327


In [49]:
##BERT SCORE EVALUATION
def evaluate_bertscore(dataset_split):
    references = []
    predictions = []

    for data in ds[dataset_split]:
        references.append(data["summary"])
        predictions.append(summarize(data["dialogue"]))

    # Compute BERTScore
    P, R, F1 = score(predictions, references, lang="en", model_type="microsoft/deberta-xlarge-mnli")

    return {
        "Precision": P.mean().item(),
        "Recall": R.mean().item(),
        "F1 Score": F1.mean().item()
    }

# Compute BERTScore for validation set
bert_scores = evaluate_bertscore("validation")
print(f"BERTScore (Validation) - Precision: {bert_scores['Precision']:.4f}, Recall: {bert_scores['Recall']:.4f}, F1 Score: {bert_scores['F1 Score']:.4f}")

BERTScore (Validation) - Precision: 0.6679, Recall: 0.7162, F1 Score: 0.6896


## TOPIC GENERATION USING DISTILGPT2


In [105]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer1 = GPT2Tokenizer.from_pretrained("distilgpt2")
model1 = GPT2LMHeadModel.from_pretrained("distilgpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [2]:
##Function to generate the topics

def generate_topic(dialogue, max_new_tokens = 20):
  prompt = f"Dialogue: {dialogue}\nTopic:"
  input_ids1 = tokenizer1.encode(prompt, return_tensors = 'pt', truncation = True, max_length=300)

  ##Generate the output
  output_ids1 = model1.generate(
      input_ids1,
      max_length = max_length,
      num_return_sequences = 1,
      pad_token_id = tokenizer.eos_token_id
  )

  ##Decode output
  generated_text = tokenizer.decode(output_ids1[0], skip_special_tokens=True)

  # Extract topic (post-processing)
  topic = generated_text.split("Topic:")[-1].strip()
  return topic

In [3]:
##Applying topic generation to the dataset
def process_dataset(dataset):
    new_topics = []
    for example in ds["train"]:  # Change to "test" or "validation" if needed
        dialogue = example["dialogue"]
        generated_topic = generate_topic(dialogue)
        new_topics.append(generated_topic)

    return new_topics

In [5]:
# generated_topics = process_dataset(ds)

In [ ]:
#  Display some generated topics
for i in range(5):
    print(f"Dialogue: {dataset['train'][i]['dialogue']}\nGenerated Topic: {generated_topics[i]}\n")